In [46]:
#Dataframe setup
import matplotlib.pyplot as plt
import numpy as np 
import math
import pandas as pd 
import openpyxl
import cvxpy as cp

In [2]:
# read main data
# it may takes a lot of time to load the data...
cols = ['Name','Sell','Buy','DIY','Source','Season/Event']
Housewares = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Housewares', usecols = cols)
Miscellaneous = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Miscellaneous', usecols = cols)
WallMounted =pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Wall-mounted', usecols = cols)
CeilingDecor = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Ceiling Decor', usecols = cols)
InteriorStructures = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Interior Structures', usecols = cols)
Wallpaper = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Wallpaper', usecols = cols)
Floors = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Floors', usecols = cols)
Rugs = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Rugs', usecols = cols)
ToolsGoods = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'ToolsGoods', usecols = cols)
Fencing = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Fencing', usecols = cols)
Tops = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Tops', usecols = cols)
Bottoms = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Bottoms', usecols = cols)
DressUp = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Dress-Up', usecols = cols)
Headwear = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Headwear', usecols = cols)
Accessories = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Accessories', usecols = cols)
Socks = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Socks', usecols = cols)
Shoes = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Shoes', usecols = cols)
Bags = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Bags', usecols = cols)
Umbrellas = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Umbrellas', usecols = cols)
ClothingOther = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Clothing Other', usecols = cols)
Insects = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Insects')
Fish = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Fish')
SeaCreatures = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Sea Creatures')
Fossils = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Fossils',usecols = ['Sell'])
Other = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Other', usecols = cols)
Recipes = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Recipes')
Seasons = pd.read_excel('Data Spreadsheet for Animal Crossing New Horizons.xlsx',sheet_name = 'Seasons and Events')

In [3]:
# read time data defined by our team
time = pd.read_excel('type-time.xlsx')

In [4]:
sheets = [Housewares, Miscellaneous, WallMounted, CeilingDecor, InteriorStructures, Wallpaper, Floors, Rugs, ToolsGoods, Fencing, Tops, Bottoms, DressUp, Headwear, Accessories, Socks, Shoes, Bags, Umbrellas, ClothingOther, Other]

# price ['Name','Sell','Buy','DIY','Source','Season/Event']
prices = pd.DataFrame()
for i in range(len(sheets)):
    prices = prices.append(sheets[i])

prices = prices.drop_duplicates()                       # drop duplicate items
prices = prices[pd.isnull(prices['Season/Event'])]      # remove items with special season/event (assume: normal day)
prices = prices[prices['Name']!='glowing moss']         # cannot grow in player's island
prices = prices[prices['Name']!='rusted part']          # cannot get from player's island in normal day
prices = prices[prices['Name']!='vine']                 # cannot grow in player's island
prices = prices[prices['Name']!='turnips']              # cannot grow in player's island

# get prices of diy items      
diy_prices = prices[prices['DIY'] == 'Yes']
diy_prices = diy_prices.reset_index(drop=True)
# add diy item's selling prices in diy recipe
diy = Recipes.copy()
diy = diy[(diy['Name']!= 'scattered papers') & (diy['Name']!= 'document stack')]
diy = diy.iloc[:,0:15]
diy = diy.merge(diy_prices, on ='Name')

del diy['Image']
del diy['Image SH']

diy = diy.reset_index(drop=True)

In [5]:
# get materials list that can be used to create DIY items by DIY recipe
diy_materials = list(set(Recipes['Material 1'])) + list(set(Recipes['Material 2'])) + list(set(Recipes['Material 3'])) + list(set(Recipes['Material 4'])) + list(set(Recipes['Material 5'])) + list(set(Recipes['Material 6']))
diy_materials = list(set(diy_materials))
for i in diy_materials:
    if pd.isnull(i):
        diy_materials.remove(i)
        
names = []
bells = []
sources =[]
left = []

for i in range(len(Other)):
    if Other.iloc[i]['Name'] in diy_materials:
        names.append(Other.iloc[i]['Name'])
        bells.append(Other.iloc[i]['Sell'])
        sources.append(Other.iloc[i]['Source'])
        
materials = pd.DataFrame(names)
materials['Sell'] = bells
materials['Source'] = sources
materials.columns = ['Name','Sell','Source']

flowers = ['weeds','cosmos','hyacinths','lilies','mums','pansies','roses','tulips','windflowers']
fruits =['apple','cherry','orange','pear','peach']
rocks = ['iron nugget','stone','clay','gold nugget']
earth =['fossil','manila clam']
vegetables =['tomato', 'potato', 'carrot', 'wheat', 'sugarcane', 'pumpkin']
nuts = ['acorn','pine cone']

# return item type list for all items in given dataframe
def get_type(df):
    item_type = []
    for i in range(len(df)):
        item = df.iloc[i]['Name']
        last_word = item.split()[-1]
        if item[-4:]=='wood' or item[-5:] == 'piece': 
            item_type.append('woods') 
        elif item in fruits:
            item_type.append('fruit')
        elif last_word in flowers:
            item_type.append('flower')
        elif item == earth:
            item_type.append('dig')
        elif item in rocks:
            item_type.append('rock')
        elif last_word == 'tree':
            item_type.append('tree')
        elif last_word in vegetables:
            item_type.append('vegetable')
        elif item == 'coconut':
            item_type.append('coconut')
        elif item in nuts:
            item_type.append('nut')
        elif item == 'pearl':
            item_type.append('pearl')
        else:
            item_type.append('other')
    return item_type

# assign item type by get_type() function
materials['Type'] = get_type(materials)             
materials = pd.merge(materials, time, on = 'Type')

In [6]:
# Get recipe for one DIY item by its index.
def get_recipe(i,df):
    quantity = ['#1','#2','#3','#4','#5','#6']
    material_num = ['Material 1','Material 2','Material 3','Material 4','Material 5','Material 6']
    
    recipe = {}
    isvalid = True          # boolean. True if item accessable
    
    for j in range(6):
        temp_q = df[quantity[j]][i]
        temp_m = df[material_num[j]][i]
        if not pd.isnull(temp_m):
            
            # check if material is accesiable without special condition
            if temp_m in list(prices['Name']):
                if temp_m not in fruits:
                    isNFS = (prices[(prices['Name']== temp_m)]['Buy'] == 'NFS').bool()
                    isGulliver = (prices[(prices['Name']== temp_m)]['Source'] != 'Gulliver').bool()
                    isGullivarrr = (prices[(prices['Name']== temp_m)]['Source'] != 'Gullivarrr').bool()
                    isCyrus = (prices[(prices['Name']== temp_m)]['Source'] != 'Cyrus').bool()
                    
                    if not (isNFS and isGulliver and isGullivarrr and isCyrus):
                        isvalid = False       
            else:
                isvalid = False
            
            # add temp material into recipe if itself is not a DIY item
            if temp_m not in set(df['Name']):
                recipe[temp_m] = temp_q
            
            else: 
                # the material itself is a DIY item
                # get recipe for this material and add the sub-material list to the result recipe list
                new_i = df.loc[df['Name']== temp_m].index[0]
                temp_recipe = get_recipe(new_i,df)[0]
                if not get_recipe(new_i,df)[1]:
                    isvalid = get_recipe(new_i,df)[1]
                for key in temp_recipe.keys():
                    if key in recipe:
                        new_q = recipe[key]+ temp_recipe[key]
                        recipe[key] = new_q
                    else:
                        recipe[key] = temp_recipe[key]
                        
    return [recipe,isvalid]

In [7]:
# get time of creating DIY items based on recipe
diy_time = []
diy_base_time = float(time[time['Type']=='diy']['Time'])    # the diy procedure itme
for i in range(len(diy)):
    recipe = get_recipe(i,diy)[0]           # get recipe by get_recipe() function
    isValid = get_recipe(i,diy)[1]          # get boolean if the item is accessiable by get_recipe() function
    
    i_time = diy_base_time                  
    
    if isValid:
        for j in recipe:
            # add time of getting materials in DIY recipe
            i_time = i_time + recipe[j] * float(materials[materials['Name'] == j]['Time'])
        diy_time.append(i_time)
    else:
        # inaccessiable item
        diy_time.append(-1)
diy['Time'] = diy_time
diy = diy[diy['Time'] > 0]      # filter with accessiable items

In [8]:
# assign item_type = 'item' for all instances in diy dataframe
diy_df = diy.copy()[['Name','Sell','Time']]
diy_df['Type'] = ['diy']*len(diy_df)

In [9]:
# return list of all possible fish type by given hour
def get_fish_type(hr):
    fish_type = ['Allday']
    if (hr > 4) and (hr <= 21):
        fish_type.append('4AM–9PM')
        if (hr <= 16) and (hr > 9):
            fish_type.append('9AM–4PM;9PM–4AM')
            fish_type.append('9AM–4PM')
        else:
            fish_type.append('4PM–9AM')
    else:
        fish_type.append('4PM–9AM')
        fish_type.append('9AM–4PM;9PM–4AM')
        fish_type.append('9PM–4AM')
        
    return fish_type
        
# return list of all possible insect type by given hour
def get_insect_type(hr):
    insect_type = ['Allday']
    if (hr > 4) and (hr <= 19):
        insect_type.append('4AM–7PM')
        if (hr >=8):
            insect_type.append('8AM–7PM')
            if (hr <= 17):
                insect_type.append('8AM–5PM')
            else: 
                insect_type.append('4AM–8AM;5PM–7PM')
                insect_type.append('5PM–4AM')
                insect_type.append('5PM–8AM')
            if (hr >= 16):
                insect_type.append('4AM–8AM;4PM–7PM')
                insect_type.append('4AM–11PM')
            else:
                insect_type.append('11PM–4PM')
        else:
            insect_type.append('4AM–8AM;4PM–7PM')
            insect_type.append('4AM–8AM;5PM–7PM')
            insect_type.append('11PM–8AM')
            insect_type.append('11PM–4PM')
    else:
        insect_type.append('5PM–4AM')
        insect_type.append('5PM–8AM')
        insect_type.append('7PM–8AM')
        if (hr < 23):
            insect_type.append('4PM–11PM')
        else:
            insect_type.append('11PM–8AM')
            insect_type.append('11PM–4PM') 
    return insect_type

# return list of all possible sea creature type by given hour
def get_sea_type(hr):
    sea_type = ['Allday']
    if (hr > 4) and (hr <= 21):
        sea_type.append('4AM–9PM')
        if (hr > 9) and (hr <= 16):
            sea_type.append('9AM–4 PM;9PM–4AM')
        else:
            sea_type.append('4PM–9AM')
    else:
        sea_type.append('4PM–9AM')
        sea_type.append('9PM–4AM')
        sea_type.append('9AM–4PM;9PM–4AM')
    
    return sea_type

In [10]:
# remove special spaces in dataframe (e.g. \xa)
# area_month = the usr defined month
# df = dataframe to clean
def clean_df (area_month, df):
    for i in range(len(df)):
        if not pd.isnull(df[area_month][i]):
            df[area_month][i] = ''.join(df[area_month][i].split())
    return df

# get data of fossil with average price from original Fossils sheet
# time = dataframe of time information
# return new fossil dataframe ['Name','Sell','Time','Type']
def fossilData(time, Fossils):   
    # time of digging a fossil = time of identify fossil in museum + time of dig fossil from the ground
    fossil_time = float(time[time['Type'] == 'identify']['Time']) + float(time[time['Type'] == 'dig']['Time'])
    # player cannot know fossil price before they identify the fossil, thus use average price here
    fossil_price = round(Fossils['Sell'].mean())
    fossil_df = pd.DataFrame([['identified fossil',fossil_price,fossil_time,'fossil']])
    fossil_df.columns = ['Name','Sell','Time','Type']
    return fossil_df

# get data of insects in given month and time from original Insects sheet
# time = dataframe of time information
# return new insect dataframe ['Name','Sell','Time','Type']
def insectData (hr_now, area_month, time, Insects): 
    insect_type = get_insect_type(hr_now)                       # get insect type list by get_sea_type() function
    insect_temp = clean_df(area_month, Insects.copy())

    # append data of all types in list
    insect_now = insect_temp[insect_temp[area_month] == insect_type[0]]
    for i in range(len(insect_type)-1):
        insect_now = insect_now.append(insect_temp[insect_temp[area_month]==insect_type[i+1]])

    insect_df = pd.DataFrame([['insect',round(insect_now['Sell'].mean()),float(time[time['Type']=='insect']['Time']),'insect']])
    insect_df.columns = ['Name','Sell','Time','Type']
    return insect_df

# get data of fish in given month and time from original Fish sheet
# time = dataframe of time information
# return new fish dataframe ['Name','Sell','Time','Type']
def fishData (hr_now, area_month, time, Fish):
    fish_type = get_fish_type(hr_now)                           # get fish type list by get_sea_type() function
    fish_temp = clean_df(area_month, Fish.copy())

    # append data of all types in list
    fish_now = fish_temp[fish_temp[area_month] == fish_type[0]]
    for i in range(len(fish_type)-1):
        fish_now = fish_now.append(fish_temp[fish_temp[area_month]==fish_type[i+1]])

    fish_df = pd.DataFrame([['fish',round(fish_now['Sell'].mean()),float(time[time['Type']=='fish']['Time']),'fish']])
    fish_df.columns = ['Name','Sell','Time','Type']
    return fish_df

# get data of sea creatures in given month and time from original SeaCreatures sheet
# time = dataframe of time information
# return new sea creatures dataframe ['Name','Sell','Time','Type']
def seaData (hr_now, area_month, time, SeaCreatures):
    # get data of sea creatures in given month and time
    sea_type = get_sea_type(hr_now)                             # get sea creatures type list by get_sea_type() function
    sea_temp = clean_df(area_month, SeaCreatures.copy())

    # append data of all types in list
    sea_now =sea_temp[sea_temp[area_month] == sea_type[0]]
    for i in range(len(sea_type)-1):
        sea_now = sea_now.append(sea_temp[sea_temp[area_month]==sea_type[i+1]])     

    sea_df = pd.DataFrame([['sea creature',round(sea_now['Sell'].mean()),float(time[time['Type']=='sea']['Time']),'sea']])
    sea_df.columns = ['Name','Sell','Time','Type']
    return sea_df

In [11]:
# Get user input
# return list = [month, area, hour_time, time_limit]
def usrInput ():
    # let user input current month and the hemisphere of the island
    # these variable will affect type of fish, insects, and sea creatures
    month = input('Please enter current month (Jan - Dec): ')
    area = input('Which Hemisphere are you in (NH/SH): ')

    # let user input starting time and the playing time limit
    hour_time = input('Please enter current time (hh:mm)(24h): ')
    time_limit = input('How long you want to play (hh:mm)(24h): ')
    return [month, area, hour_time, time_limit]

# get parameters from list = [month, area, hour_time, time_limit]
# return list = [s_limit, area_month, hr_now, min_now, hr_limit, min_limit, hr_end, min_end]
def getParam (in_list):
    area_month = in_list[1] + ' '+ in_list[0]
    
    hr_now = int(in_list[2].split(':')[0])
    min_now =int(in_list[2].split(':')[1])
    hr_limit = int(in_list[3].split(':')[0])
    min_limit = int(in_list[3].split(':')[1])

    s_limit = hr_limit * 3600 + min_limit * 60          # time limit in seconds

    # calculate the end hour and minute
    hr_end = hr_now + hr_limit
    min_end = min_now + min_limit
    if (min_limit + min_now) > 60:
        hr_end = hr_end + 1
        min_end = min_end - 60
    
    return [s_limit, area_month, hr_now, min_now, hr_limit, min_limit, hr_end, min_end]


In [18]:
# merged dataframe for all accessible items
# return dataframe of all items
def itemData (materials, diy_df, fossil_df, fish_df, insect_df, sea_df):
    item_df = materials[['Name','Sell','Time','Type']]
    item_df = item_df.append(diy_df[['Name','Sell','Time','Type']])
    item_df = item_df.append(fish_df)
    item_df = item_df.append(insect_df)
    item_df = item_df.append(sea_df)
    item_df = item_df.append(fossil_df)
    item_df = item_df.dropna()
    item_df = item_df.reset_index(drop=True)
    
    # item_df.to_excel('items.xlsx',index=False,header=True)
    return item_df

In [13]:
# return number of area unit required for farming the item in given type
# type_str = item type
# df = dataframe (items.xlsx)
# n = list of item quantity
def get_source_number(type_str, df, n):
    # get item index list of given item type
    index = df[df['Type']==type_str].index.values
    num = 0
    divide = 1
    
    if type_str =='fruit':
        for i in index:
            # one fruit tree can only produce one type of furit
            num = num + n[i]/3
    else:
        for i in index:
            num = num + n[i]
        if type_str =='woods':
            # Player can get three woods type items from a tree, a tree takes one unit area;
            divide = 3
        elif type_str =='rock':
            # Player can get seven rock type items from a rock, a rock takes one unit area;
            divide = 7
        elif type_str =='coconut':
            # Player can get two coconutsfrom a coconut tree, a coconut tree takes one unit area;
            divide = 2

        num = num/divide
    print(num)

    return num


# return number of bag grids required for item in given type
# type_str = item type
# df = dataframe (items.xlsx)
# n = list of item quantity
def grid_num(type_str, df, n):
    # get item index list of given item type
    index = df[df['Type']==type_str].index.values
    num = 0

    # One grid can only contain on items in same type
    # e.g. 1 oranges and 1 apples take 2 grids
    # each normal item take 1 grid, such as DIY, creatures, etc.
    divide = 1

    if type_str =='fruit' or type_str =='vegetable':
        # 10 fruits or vegetables in same type take 1 grid
        divide = 10
    elif type_str =='woods' or type_str =='rock':
        # 30 woods or rocks in same type take 1 grid
        divide = 30
    for i in index:
        num = num + n[i]
    return num/divide

In [69]:
# build model with time (dataframe), item_df (dataframe), and s_limit (int)
# return list = [objective function value, variable list]

def buildModel (time, item_df, s_limit):
    # build model
    n = cp.Variable(len(item_df), integer = True)       # variable list: item quantity (interger)
    constraints = []
    st_info = []        # label list for constraints

    # assumptions
    # areas shows below is calculated based on Hui's island area. Data may be a little bit for other island.
    land_area = 7560
    beach_area = 1144
    building_area = 408
    rock_area = 35                                    
    net_area = land_area - building_area-rock_area      # net area = total land area for farming

    types = list(set(time['Type']))                     # get the item types as list from time table

    # total number of rocks <= 6
    constraints.append(get_source_number('rock',item_df,n) <= 6)
    st_info.append('# of rock')  

    # total land area of farming target item <= net area
    # get token area of each types of item by function get_source_number()
    # each tree takes 1 unit, but planting a tree requires 8 units empty around it.
    constraints.append(9*(get_source_number('woods',item_df,n))+get_source_number('fossil',item_df,n)+get_source_number('vegetable',item_df,n)   +get_source_number('flower',item_df,n)<= net_area)
    st_info.append('net land area')

    # total beach area of planting coconut <= beach area
    # only coconut tree can be planted in beach area. Similar as trees, it require total of 9 units for planting.
    constraints.append(9 * (get_source_number('coconut',item_df,n))<= beach_area)
    st_info.append('beach area')

    # coconut tree and fruit tree can also produce three woods per tree.
    # total number of coconut tree and fruit tree <= total number of woods
    constraints.append( get_source_number('coconut',item_df,n) + get_source_number('fruit',item_df,n)-get_source_number('woods',item_df,n)<=0)
    st_info.append('fruit trees')

    sum_grid = 0
    for i in range(len(types)):
        temp_grid = grid_num(types[i],item_df,n)        # get number of token grid number for item by grid_num() function
        sum_grid = sum_grid + temp_grid                 # sum total required grid number in bag
    # There are 40 grids in bag. We assume player always carry 4 basic tools for activities, which already take 4 grids in bag
    # player's bag can contain 36 grids item. 
    # The player sell items in store when the bag is full. 
    sum_time = (sum_grid/36) * float(time[time['Type']=='shop']['Time'])    # total time spend on selling items

    gold = 0
    for i in range(len(item_df)):
        # add time spend by getting each item
        temp_time = float(item_df['Time'][i])
        sum_time = sum_time + n[i]* temp_time 

        if item_df['Name'][i]=='gold nugget':
            gold = gold + n[i]
        elif 'gold' in item_df['Name'][i] or item_df['Name'][i]=='King Tut mask':
            # these are DIY items that use gold nugget as source
            # the average gold requirement is 3
            gold = gold + 3 * n[i]
        elif 'mushroom' in item_df['Name'][i]:
            # mushroom is a special item that cannot appear in a normal day
            constraints.append(n[i] == 0)
            st_info.append('mushroom')
        elif item_df['Name'][i] == 'fossil' or item_df['Name'][i]=='identified fossil':
            # At most 3 fossils can appear in one day
            constraints.append(n[i] <= 3)
            st_info.append('fossil')
        # post analysis
        # elif item_df['Name'][i] == 'iron nugget':
        #     constraints.append(n[i] ==0)
        # elif item_df['Name'][i] == 'blue rose crown':
        #     constraints.append(n[i] ==0)
        # elif item_df['Name'][i] == 'blue rose wreath':
        #     constraints.append(n[i] ==0)

    constraints.append(gold <= 1)                       # gold nugget quantity <= 1
    st_info.append('gold')

    constraints.append(sum_time <= s_limit)             # total time spend <= playing time limit
    st_info.append('time')


    obj_func = 0        # objective function
    for i in range(len(item_df)):
        temp_price = float(item_df['Sell'][i])         # get item price from dataframe
        obj_func =  obj_func + n[i] * temp_price       # obj_func = SUM(item_quantity * item_price)
        constraints.append(n[i] >= 0)                  # positivity
        st_info.append('positive')
    problem=cp.Problem(cp.Maximize(obj_func),constraints)   # profit maximization
    problem.solve(solver=cp.GUROBI,verbose = True)          # solve by GUROBI

    return [obj_func.value, n]

In [27]:
# print our the user input information from input_list
# input_list = [month, area, hour_time, time_limit]
def printUsrParam (input_list):
    # print out information
    print('Month: ', input_list[0])
    print('Hemesiphere (NH/SH):', input_list[1])
    print('start time =', input_list[2])
    print('total playing hours:', input_list[3])

# extract item infomation from source data and summarize the result from model
# return summarized result dataframe
def getResultDf (result, item_df):
    # print item table with name, quantity, and price (game money: Bells)
    name_list = []
    quantity_list = []
    bell_list = []
    for i in range(len(item_df)):
        if result[1][i].value > 0:
            name_list.append(item_df.iloc[i]['Name'])
            quantity_list.append(int(result[1][i].value))
            bell_list.append(item_df.iloc[i]['Sell'] * int(result[1][i].value))
    summary = pd.DataFrame()
    summary['Name'] = name_list
    summary['Quantity'] = quantity_list
    summary['Bell'] = bell_list

    print('The maximize Bells =',result[0])
    print('The item are:')
    return summary


In [70]:
# run program 

# our sample input = ['Nov','NH','13:10','0:30'] 
input_list = usrInput()                 # user input = [month, area, hour_time, time_limit]
param_list = getParam(input_list)       # [s_limit, area_month, hr_now, min_now, hr_limit, min_limit, hr_end, min_end]

# get key parameters from list
hour_time = input_list[2]
time_limit = input_list[3]
hr_now = param_list[2]
s_limit = param_list[0]
area_month = param_list[1]

# get dataframe by self-defined functions
fossil_df = fossilData(time, Fossils)
fish_df = fishData(hr_now, area_month, time, Fish)
insect_df = insectData(hr_now, area_month, time, Insects)
sea_df = seaData(hr_now, area_month, time, SeaCreatures)

# DataFrame of materials and diy_df are preprocessed in previous steps
item_df = itemData(materials, diy_df, fossil_df, fish_df, insect_df, sea_df)

# build model
result = buildModel(time, item_df, s_limit)     # build model by function buildModel()
# print model result
printUsrParam(input_list)                       # print user defined parameters by function printUsrParam()
print(getResultDf(result, item_df))             # print the summarized result table by function getResultDf()

(var51682638[136] + var51682638[137] + var51682638[138] + var51682638[139]) / 7.0
(var51682638[69] + var51682638[70] + var51682638[71] + var51682638[72]) / 3.0
var51682638[715] / 1.0
(var51682638[127] + var51682638[128] + var51682638[129] + var51682638[130] + var51682638[131] + var51682638[132] + var51682638[133] + var51682638[134] + var51682638[135]) / 1.0
(var51682638[73] + var51682638[74] + var51682638[75] + var51682638[76] + var51682638[77] + var51682638[78] + var51682638[79] + var51682638[80] + var51682638[81] + var51682638[82] + var51682638[83] + var51682638[84] + var51682638[85] + var51682638[86] + var51682638[87] + var51682638[88] + var51682638[89] + var51682638[90] + var51682638[91] + var51682638[92] + var51682638[93] + var51682638[94] + var51682638[95] + var51682638[96] + var51682638[97] + var51682638[98] + var51682638[99] + var51682638[100] + var51682638[101] + var51682638[102] + var51682638[103] + var51682638[104] + var51682638[105] + var51682638[106] + var51682638[107] + v